In [73]:
import librosa
import os
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

In [3]:
import tensorflow as tf
tf.__version__

from tensorflow.python.client import device_lib
device_lib.list_local_devices() # GPU 사용

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9597750160518277546
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1968491724
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7853287511377922102
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 3GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
 xla_global_id: 416903419]

In [56]:
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import random

In [41]:
CFG = {
    'SR':44100,   # 음원의 Samplerate
    'N_MFCC':128, # MFCC Vector 개수 (음파를 벡터로 변환)
    'SEED':2023
}

In [42]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

In [9]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [10]:
train

,SAMPLE_ID,SAMPLE_PATH,FAN_TYPE,LABEL
0,TRAIN_0000,./train/TRAIN_0000.wav,2,0
1,TRAIN_0001,./train/TRAIN_0001.wav,0,0
2,TRAIN_0002,./train/TRAIN_0002.wav,0,0
3,TRAIN_0003,./train/TRAIN_0003.wav,2,0
4,TRAIN_0004,./train/TRAIN_0004.wav,2,0
...,...,...,...,...
1274,TRAIN_1274,./train/TRAIN_1274.wav,2,0
1275,TRAIN_1275,./train/TRAIN_1275.wav,2,0
1276,TRAIN_1276,./train/TRAIN_1276.wav,2,0
1277,TRAIN_1277,./train/TRAIN_1277.wav,2,0


In [13]:
test

,SAMPLE_ID,SAMPLE_PATH,FAN_TYPE
0,TEST_0000,./test/TEST_0000.wav,2
1,TEST_0001,./test/TEST_0001.wav,2
2,TEST_0002,./test/TEST_0002.wav,0
3,TEST_0003,./test/TEST_0003.wav,0
4,TEST_0004,./test/TEST_0004.wav,0
...,...,...,...
1509,TEST_1509,./test/TEST_1509.wav,0
1510,TEST_1510,./test/TEST_1510.wav,2
1511,TEST_1511,./test/TEST_1511.wav,0
1512,TEST_1512,./test/TEST_1512.wav,0


In [77]:
def get_mfcc(data):
    with tf.device("/device:GPU:0"):
        features = []
        for f in tqdm(data['SAMPLE_PATH'].apply(lambda x: './data'+x[1:])):
            # wav 파일 로드
            y, sr = librosa.load(f, sr=CFG['SR']) # y : 벡터값, sr : Samplerate
            
            # MFCC Vector 추출 <= 128까지만 추출됨
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
            
            features.append([np.mean(e) for e in mfcc])
            
        return features

In [78]:
train_feat = get_mfcc(train)

100%|██████████████████████████████████████████████████████████████████████████████| 1279/1279 [06:29<00:00,  3.29it/s]


In [58]:
test_feat = get_mfcc(test)

100%|██████████████████████████████████████████████████████████████████████████████| 1514/1514 [09:12<00:00,  2.74it/s]


In [74]:
# Parameter 설정
input_dim = CFG['N_MFCC']
encoding_dim = 20

input_layer = Input(shape=(input_dim, ))

# Autoencoder : 정보를 압축시켰다가 다시 확장하며 입력받은 것과 최대한
# 비슷한 결과값을 반환하는 '재구성'의 방법을 통해 특정 데이터의 패턴을 인코딩

# Model Layer 설정

# ===== Encoding Layer =====
encoder1 = Dense(encoding_dim, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder1 = Dropout(0.3)(encoder1)
encoder2 = Dense(int(encoding_dim / 2), activation='relu')(encoder1)
encoder2 = Dropout(0.3)(encoder2)

# ===== Decoding Layer =====
decoder1 = Dense(int(encoding_dim / 2), activation='tanh')(encoder2)
decoder1 = Dropout(0.3)(decoder1)
decoder2 = Dense(input_dim, activation='relu')(decoder1)

# Model 구조 설정
autoencoder = Model(inputs=input_layer, outputs=decoder2)

In [84]:
# 
EPOCHS = 250
batch_size = 16
autoencoder.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['binary_accuracy'])

checker = ModelCheckpoint(filepath='AE_model.h5',
                          verbose=0,
                          save_best_only=True)

tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)

with tf.device("/device:GPU:0"):
    history = autoencoder.fit(np.array(train_feat), train['LABEL'],
                              epochs = EPOCHS,
                              batch_size = batch_size,
                              shuffle=True,
#                               validation_data=(tr_val, tar_val),
                              verbose=1,
                              callbacks=[checker, tensorboard]).history

Epoch 1/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 2/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 3/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 4/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 5/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 6/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 7/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 8/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 9/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 10/250
80/80 [==========================

Epoch 31/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 32/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 33/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 34/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 35/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 36/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 37/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 38/250
80/80 [==============================] - 1s 9ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 39/250
80/80 [==============================] - 1s 9ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 40/250
80/80 [=================

Epoch 61/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 62/250
80/80 [==============================] - 1s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 63/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 64/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 65/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 66/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 67/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 68/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 69/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 70/250
80/80 [=================

Epoch 91/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 92/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 93/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 94/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 95/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 96/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 97/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 98/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 99/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 100/250
80/80 [================

Epoch 121/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 122/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 123/250
80/80 [==============================] - 1s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 124/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 125/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 126/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 127/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 128/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 129/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 130/250
80/80 [=======

Epoch 151/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 152/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 153/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 154/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 155/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 156/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 157/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 158/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 159/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 160/250
80/80 [=======

Epoch 181/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 182/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 183/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 184/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 185/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 186/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 187/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 188/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 189/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 190/250
80/80 [=======

Epoch 211/250
80/80 [==============================] - 1s 8ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 212/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 213/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 214/250
80/80 [==============================] - 1s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 215/250
80/80 [==============================] - 1s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 216/250
80/80 [==============================] - 1s 7ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 217/250
80/80 [==============================] - 1s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 218/250
80/80 [==============================] - 0s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 219/250
80/80 [==============================] - 1s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 220/250
80/80 [=======

Epoch 241/250
80/80 [==============================] - 1s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 242/250
80/80 [==============================] - 1s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 243/250
80/80 [==============================] - 1s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 244/250
80/80 [==============================] - 1s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 245/250
80/80 [==============================] - 0s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 246/250
80/80 [==============================] - 0s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 247/250
80/80 [==============================] - 0s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 248/250
80/80 [==============================] - 1s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 249/250
80/80 [==============================] - 1s 6ms/step - loss: 0.0019 - binary_accuracy: 1.0000
Epoch 250/250
80/80 [=======

In [114]:
threshold = 2828 - 142 # 적절한 threshold 설정 필요

preds = autoencoder.predict(np.array(test_feat))
mse = np.mean(np.power(np.array(test_feat) - preds, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse})
error_df.describe()

48/48 [==============================] - 0s 2ms/step


,reconstruction_error
count,1514.000000
mean,2700.283447
std,142.862991
min,2370.440430
25%,2565.858582
50%,2712.258057
75%,2828.749146
max,3023.095947


In [115]:
y_pred = [1 if  i> threshold else 0 for i in error_df.reconstruction_error.values]
y_pred # 예측값

[0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,


In [116]:
test['LABEL'] = y_pred

In [117]:
submit = pd.concat([test.iloc[:,0], test.iloc[:, -1]], axis=1)
submit

,SAMPLE_ID,LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,1
3,TEST_0003,0
4,TEST_0004,1
...,...,...
1509,TEST_1509,1
1510,TEST_1510,1
1511,TEST_1511,1
1512,TEST_1512,1


In [118]:
submit.to_csv('submission_04.csv', index=False, encoding='utf-8')